In [1]:
import numpy as np
import pandas as pd
import shutil

import os,sys
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
# imp code
# check_Data_meter_Status['Duplicated'] = check_Data_meter_Status.duplicated(subset=['METERNO']).map({True: 'Duplicated', False: 'Unique'})

In [3]:
df_warehouse=pd.read_excel(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\WAREHOUSE_DATA_FILE.xlsx")
df_warehouse.to_csv("warehouse_data.csv",index=False)

warehouse_Data = pd.read_csv("C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_data.csv")

#warehouse_Data = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\WAREHOUSE_DATA_FILE.csv")

warehouse_Data.shape


(1126, 10)

In [4]:
warehouse_Data['WAREHOUSE_UTILITY'].unique()

array(['SBPDCL', 'NBPDCL'], dtype=object)

In [5]:
#warehouse_Data = warehouse_Data[warehouse_Data['WAREHOUSE_UTILITY']=='SBPDCL']
#warehouse_Data.shape

In [6]:


# changes in dataset like :- UPPER, SPACE REMOVAL , DATATYPE

warehouse_Data = warehouse_Data.astype(str)
warehouse_Data.columns = warehouse_Data.columns.str.upper()
warehouse_Data = warehouse_Data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
warehouse_Data.columns = warehouse_Data.columns.str.strip()

In [7]:
warehouse_Data.shape

(1126, 10)

In [8]:
warehouse_Data.columns

Index(['SL NO', 'METER NO.', 'PHASE', 'WAREHOUSE_UTILITY', 'WAREHOUSE_STATUS',
       'MAKE', 'IME', 'FINAL SIM', 'IMSI', 'IP'],
      dtype='object')

In [9]:
# data ingestion

NB_METER_STATUS=pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\NB_METER_STATUS.csv")
SB_METER_STATUS=pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\SB_METER_STATUS.csv")

print("DATA INGESTION DONE")
#-----------------------------------------------------------------------------------------------------------------------------------

# data correction


#-----------------------------------------------------------------------------------------------------------------------------------
NB_METER_STATUS = NB_METER_STATUS.astype(str)
NB_METER_STATUS.columns = NB_METER_STATUS.columns.str.upper()
NB_METER_STATUS = NB_METER_STATUS.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
NB_METER_STATUS.columns = NB_METER_STATUS.columns.str.strip()


#-----------------------------------------------------------------------------------------------------------------------------------

SB_METER_STATUS = SB_METER_STATUS.astype(str)
SB_METER_STATUS.columns = SB_METER_STATUS.columns.str.upper()
SB_METER_STATUS = SB_METER_STATUS.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
SB_METER_STATUS.columns = SB_METER_STATUS.columns.str.strip()



print("DATA CORRECTION COMPLETE")

DATA INGESTION DONE
DATA CORRECTION COMPLETE


In [10]:
assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_SB.CSV")
assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_NB.CSV")

In [11]:
#NB_METER_STATUS.head(1)

In [12]:
#NB_METER_STATUS.duplicated(subset=['INSTALLATIONNO'],keep=False)].sort_values(by=['INSTALLATIONNO'],ascending=True)

In [13]:
warehouse_utility_importation_demand =  input(f"""ENTER THE WAREHOUSE ON DEMAND UTILITY: 
                                                1. NBPDCL 
                                                2. SBPDCL 
                                                3. NO DEMAND""")


if warehouse_utility_importation_demand == "1":
    assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_NB.CSV")


    # changes in dataset like :- UPPER, SPACE REMOVAL , DATATYPE
    assest_initiation_invent_nb = assest_initiation_invent_nb.astype(str)
    assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.upper()
    assest_initiation_invent_nb = assest_initiation_invent_nb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
    assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.strip()

    # DATA
    NB_SB_COMBINED_DATAFRAME=assest_initiation_invent_nb



elif warehouse_utility_importation_demand == "2":
    assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_SB.CSV")

    # changes in dataset like :- UPPER, SPACE REMOVAL , DATATYPE
    assest_initiation_invent_sb = assest_initiation_invent_sb.astype(str)
    assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.upper()
    assest_initiation_invent_sb = assest_initiation_invent_sb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
    assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.strip()

    NB_SB_COMBINED_DATAFRAME=assest_initiation_invent_sb
    
elif warehouse_utility_importation_demand == "3":
    assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_SB.CSV")
    assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_NB.CSV")
    

    # changes in dataset like :- UPPER, SPACE REMOVAL , DATATYPE
    
    # Convert all columns to string data type

    warehouse_Data = warehouse_Data.astype(str)
    assest_initiation_invent_nb = assest_initiation_invent_nb.astype(str)
    assest_initiation_invent_sb = assest_initiation_invent_sb.astype(str)



    # change column name into upper string

    warehouse_Data.columns = warehouse_Data.columns.str.upper()
    assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.upper()
    assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.upper()



    # Remove leading and trailing whitespaces from all columns where dtype is 'O'

    warehouse_Data = warehouse_Data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
    assest_initiation_invent_nb = assest_initiation_invent_nb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
    assest_initiation_invent_sb = assest_initiation_invent_sb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



    # strip the spaces in column 

    warehouse_Data.columns = warehouse_Data.columns.str.strip()
    assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.strip()
    assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.strip()


    # concat the assest_initiation_invent_sb and assest_initiation_invent_nb
    NB_SB_COMBINED_DATAFRAME = pd.concat([assest_initiation_invent_sb, assest_initiation_invent_nb], ignore_index=True)


    
else:
    print("plz chose correct option")


In [14]:
warehouse_Data.count()
# SHAPE OF DATA

print(f"DATASET SHAPE : {warehouse_Data.shape[0]}")

DATASET SHAPE : 1126


In [15]:
warehouse_Data.columns

Index(['SL NO', 'METER NO.', 'PHASE', 'WAREHOUSE_UTILITY', 'WAREHOUSE_STATUS',
       'MAKE', 'IME', 'FINAL SIM', 'IMSI', 'IP'],
      dtype='object')

In [16]:
# checking is null

NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME["NETWORK_ID"].isnull()].count()

METERNUMBER                                  0
MFG_SERIAL_NUM                               0
UDC_ID                                       0
NETWORK_ID                                   0
INSTALLATIONNUMBER                           0
STATUS                                       0
INSERT_TIME                                  0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    0
dtype: int64

In [17]:
# checking duplicate value

NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME.duplicated(keep = False)]



,METERNUMBER,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,INSERT_TIME,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"


In [18]:
if NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME.duplicated(keep = False)].shape[0]==0:
    print(" NONE OF 2 ROWS ARE DUPLIUCATED")
else:
    NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME.duplicated(keep = False)]



 NONE OF 2 ROWS ARE DUPLIUCATED


In [19]:
NB_SB_COMBINED_DATAFRAME.head(1)

,METERNUMBER,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,INSERT_TIME,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
0,GOEGP4970294,GP4970294,404759210464358,2001:4490:0800:5F25:0000:0000:0000:0001,nan,Meter Shop,8/7/2019 1:26:44 PM,SBPDCL


# WORK ON INSTALLATION NO.

In [20]:
# droping not used column in the NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop(['METERNUMBER','INSERT_TIME'], axis=1, inplace=True)
print(" DROP COLUMN DONE :  'METERNUMBER','INSERT_TIME'  ")

 DROP COLUMN DONE :  'METERNUMBER','INSERT_TIME'  


In [21]:
"""
# droping not used column in the NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop(['METERNUMBER', 'INSTALLATIONNUMBER','INSERT_TIME'], axis=1, inplace=True)
print(" DROP COLUMN DONE :  'METERNUMBER', 'INSTALLATIONNUMBER','INSERT_TIME'  ")

"""

'\n# droping not used column in the NB_SB_COMBINED_DATAFRAME\n\nNB_SB_COMBINED_DATAFRAME.drop([\'METERNUMBER\', \'INSTALLATIONNUMBER\',\'INSERT_TIME\'], axis=1, inplace=True)\nprint(" DROP COLUMN DONE :  \'METERNUMBER\', \'INSTALLATIONNUMBER\',\'INSERT_TIME\'  ")\n\n'

In [22]:
print(f" SHAPE OF NB_SB_COMBINED_DATAFRAME :- {NB_SB_COMBINED_DATAFRAME.shape[0]}")

 SHAPE OF NB_SB_COMBINED_DATAFRAME :- 1886532


#### DROP DUPLICATE 

In [23]:
# dop duplicate in  NB_SB_COMBINED_DATAFRAME

#NB_SB_COMBINED_DATAFRAME.drop_duplicates(subset=['MFG_SERIAL_NUM','UDC_ID',"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], keep='first', inplace=True, ignore_index=True)

# DUPLICATE NOT DROPPING

In [24]:
print(f" SHAPE OF NB_SB_COMBINED_DATAFRAME :- {NB_SB_COMBINED_DATAFRAME.shape[0]}")

 SHAPE OF NB_SB_COMBINED_DATAFRAME :- 1886532


In [25]:
NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME.duplicated(subset=['MFG_SERIAL_NUM'], keep=False)].sort_values(by=['MFG_SERIAL_NUM'],ascending=True)

,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
517751,GL8548300,405856707172350,2405:0203:0532:43AE:0000:0000:0000:0010,5001970345,Refurbished,SBPDCL
250498,GL8548300,405856706536659,2405:0203:0528:DBEF:0000:0000:0000:0010,5001970345,Refurbished,SBPDCL
981200,GL8548303,405856707299383,2405:0203:0531:9328:0000:0000:0000:0010,5004056701,Installed,SBPDCL
480801,GL8548303,405856706536685,2405:0203:0528:DBF2:0000:0000:0000:0010,5004056701,Installed,SBPDCL
13877,GL8548303,405856707299383,2405:0203:0531:9328:0000:0000:0000:0010,5002973847,Installed,SBPDCL
...,...,...,...,...,...,...
899382,LT8057018,405856706103170,2405:0203:0522:C558:0000:0000:0000:0010,5001720263,Installed,SBPDCL
866000,LT8057018,405856707247502,2405:0203:0532:1F7A:0000:0000:0000:0010,5002128109,Installed,SBPDCL
232561,LT8057018,405856706103170,2405:0203:0522:C558:0000:0000:0000:0010,5002128109,Installed,SBPDCL
734098,LT8066321,405856707025535,2405:0203:0534:0B4C:0000:0000:0000:0010,5003101279,Installed,SBPDCL


In [26]:
NB_SB_COMBINED_DATAFRAME.head(1)

,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
0,GP4970294,404759210464358,2001:4490:0800:5F25:0000:0000:0000:0001,nan,Meter Shop,SBPDCL


# CHANGES IN  NB_SB_COMBINED_DATAFRAME :- DATATYPE,UPPER,SPACES

In [27]:


# Convert all columns to string data type

NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.astype(str)


NB_SB_COMBINED_DATAFRAME.columns = NB_SB_COMBINED_DATAFRAME.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'
NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


# JOIN



#### METER

In [28]:
# here first merge with 2 column   meter no and utility   and the if some rows are null then merge those rows only only with meter no. 

In [29]:
#if ((warehouse_Data['METER NO.']== NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']) and (warehouse_Data['WAREHOUSE_UTILITY']== NB_SB_COMBINED_DATAFRAME["DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"])):
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['MFG_SERIAL_NUM'].isnull()]
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_DROPED_DATA = (LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.dropna(subset=['MFG_SERIAL_NUM','UDC_ID']))

if LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL.shape[0] > 0:
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL[warehouse_Data.columns]
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK=pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.'], right_on=['MFG_SERIAL_NUM'], how='left')
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.concat([LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_DROPED_DATA, LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK], ignore_index=True)
else:
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA


    


In [30]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.head(2)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL


In [31]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['MFG_SERIAL_NUM'].isnull()].shape[0]

3

In [32]:
#NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']=='GP4465021']

In [33]:


#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')


In [34]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.count()

SL NO                                        1126
METER NO.                                    1126
PHASE                                        1126
WAREHOUSE_UTILITY                            1126
WAREHOUSE_STATUS                             1126
MAKE                                         1126
IME                                          1126
FINAL SIM                                    1126
IMSI                                         1126
IP                                           1126
MFG_SERIAL_NUM                               1123
UDC_ID                                       1123
NETWORK_ID                                   1123
INSTALLATIONNUMBER                           1123
STATUS                                       1123
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1123
dtype: int64

In [35]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.csv',index=False)

#### IMSI

In [36]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['IMSI'], right_on=['UDC_ID'], how='left')

In [37]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

SL NO                                        1180
METER NO.                                    1180
PHASE                                        1180
WAREHOUSE_UTILITY                            1180
WAREHOUSE_STATUS                             1180
MAKE                                         1180
IME                                          1180
FINAL SIM                                    1180
IMSI                                         1180
IP                                           1180
MFG_SERIAL_NUM                               1180
UDC_ID                                       1180
NETWORK_ID                                   1180
INSTALLATIONNUMBER                           1180
STATUS                                       1180
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1180
dtype: int64

In [38]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()]

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"


# duplicate row while we join with imsi 

In [39]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[~LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()].count()

In [40]:
# duplicated row ..  bcz in asset initiation file there are 2 installtion no.  with respect to 1 imsi no.

duplicate_check = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].duplicated(keep=False)]

duplicate_check.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv',index=False)
duplicate_check.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv",index=False)

In [41]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()


SL NO                                        1180
METER NO.                                    1180
PHASE                                        1180
WAREHOUSE_UTILITY                            1180
WAREHOUSE_STATUS                             1180
MAKE                                         1180
IME                                          1180
FINAL SIM                                    1180
IMSI                                         1180
IP                                           1180
MFG_SERIAL_NUM                               1180
UDC_ID                                       1180
NETWORK_ID                                   1180
INSTALLATIONNUMBER                           1180
STATUS                                       1180
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1180
dtype: int64

In [42]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.drop_duplicates(subset=['IMSI'],inplace=True)


In [43]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

SL NO                                        1126
METER NO.                                    1126
PHASE                                        1126
WAREHOUSE_UTILITY                            1126
WAREHOUSE_STATUS                             1126
MAKE                                         1126
IME                                          1126
FINAL SIM                                    1126
IMSI                                         1126
IP                                           1126
MFG_SERIAL_NUM                               1126
UDC_ID                                       1126
NETWORK_ID                                   1126
INSTALLATIONNUMBER                           1126
STATUS                                       1126
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1126
dtype: int64

#### IP

In [44]:
#  JOIN COMBINED DATA AND WAREHOUSE DATA

LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['IP'], right_on=['NETWORK_ID'], how='left')




# changhe name 
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns = {"MFG_SERIAL_NUM":"IP_MATCHED_METER_NO","UDC_ID": "IP_MATCH_UDC_ID","NETWORK_ID":"IP_MATCHED_IP","STATUS":"IP_MATCHED_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"IP_MATCHED_UTILITY"},
    inplace=True
)




# COLUMN

LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_COLUMN=LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA.columns
#LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_COLUMN
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_COLUMN]

In [45]:
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,INSTALLATIONNUMBER,IP_MATCHED_STATUS,IP_MATCHED_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL


In [46]:
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.duplicated(subset=['IP_MATCHED_IP'],keep=False)].sort_values(by=['IP_MATCHED_IP'],ascending=False)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,INSTALLATIONNUMBER,IP_MATCHED_STATUS,IP_MATCHED_UTILITY
1138,1097,GL8995074,1 Phase,NBPDCL,S2S repaired,GENUS 4G,862058062330373,8991856040708255378,405856708800382,2405:0203:0534:CB37:0000:0000:0000:0010,GL8611463,405856708800382,2405:0203:0534:CB37:0000:0000:0000:0010,5000553001,Installed,SBPDCL
1139,1097,GL8995074,1 Phase,NBPDCL,S2S repaired,GENUS 4G,862058062330373,8991856040708255378,405856708800382,2405:0203:0534:CB37:0000:0000:0000:0010,GL8611463,405856708800382,2405:0203:0534:CB37:0000:0000:0000:0010,5002130086,Installed,SBPDCL
1182,1126,GL8998210,1 Phase,NBPDCL,S2S repaired,GENUS 4G,862058062331967,8991856040707048645,405856707593640,2405:0203:0533:9FA6:0000:0000:0000:0010,GL8953657,405856707593640,2405:0203:0533:9FA6:0000:0000:0000:0010,5004034894,Installed,SBPDCL
1181,1126,GL8998210,1 Phase,NBPDCL,S2S repaired,GENUS 4G,862058062331967,8991856040707048645,405856707593640,2405:0203:0533:9FA6:0000:0000:0000:0010,GL8953657,405856707593640,2405:0203:0533:9FA6:0000:0000:0000:0010,5001958300,Installed,SBPDCL
1053,1033,Gl8891087,1 Phase,SBPDCL,S2S repaired,GENUS 4G,862058061535634,8991856040706278018,405856706823027,2405:0203:0533:708B:0000:0000:0000:0010,GL8941598,405856706823027,2405:0203:0533:708B:0000:0000:0000:0010,BTABT-9327,Installed,SBPDCL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,61,LT0248258,1 Phase,SBPDCL,S2S repaired,L&T,869810056244504,8991856050706337157,405856706337157,2405:0203:0523:FB4E:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN,NaN
850,851,LT0240239,1 Phase,SBPDCL,S2S repaired,L&T,869810055979449,8991856050706324281,405856706324281,2405:0203:0523:DBFB:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN,NaN
851,852,LT0240477,1 Phase,SBPDCL,S2S repaired,L&T,869810055373247,8991856050706352344,405856706352344,2405:0203:0523:DCE9:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN,NaN
855,856,LT0240237,1 Phase,SBPDCL,S2S repaired,L&T,869810056258777,8991856050706353804,405856706353804,2405:0203:0523:DBF9:0000:0000:0000:0010,NaN,NaN,NaN,NaN,NaN,NaN


##### joining both the table "   LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA   "  ,  " LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA "  and    " LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA "

In [47]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [48]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns={"UDC_ID": "METER_MATCH_UDC_ID", "NETWORK_ID": "METER_MATCH_IP", "STATUS": "METER_MATCH_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"METER_MATCH_UTILITY"},
    inplace=True
)



In [49]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column in modified column

In [50]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN

Index(['SL NO', 'METER NO.', 'PHASE', 'WAREHOUSE_UTILITY', 'WAREHOUSE_STATUS',
       'MAKE', 'IME', 'FINAL SIM', 'IMSI', 'IP', 'MFG_SERIAL_NUM',
       'METER_MATCH_UDC_ID', 'METER_MATCH_IP', 'INSTALLATIONNUMBER',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY'],
      dtype='object')

In [51]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN]

In [52]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL


In [53]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED['METER_MATCH_UDC_ID']=='nan'].count()

SL NO                  0
METER NO.              0
PHASE                  0
WAREHOUSE_UTILITY      0
WAREHOUSE_STATUS       0
MAKE                   0
IME                    0
FINAL SIM              0
IMSI                   0
IP                     0
MFG_SERIAL_NUM         0
METER_MATCH_UDC_ID     0
METER_MATCH_IP         0
INSTALLATIONNUMBER     0
METER_MATCH_STATUS     0
METER_MATCH_UTILITY    0
dtype: int64

In [54]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"METER_MATCH_UDC_ID","METER_MATCH_IP","METER_MATCH_STATUS","METER_MATCH_UTILITY"]]

In [55]:
# LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [56]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns = {"MFG_SERIAL_NUM":"IMSI_MATCHED_METER_NO","NETWORK_ID":"IMSI_MATCHED_IP","STATUS":"IMSI_MATCHED_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"IMSI_MATCHED_UTILITY"},
    inplace=True
)

In [57]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,INSTALLATIONNUMBER,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL


In [58]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN=LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN

Index(['SL NO', 'METER NO.', 'PHASE', 'WAREHOUSE_UTILITY', 'WAREHOUSE_STATUS',
       'MAKE', 'IME', 'FINAL SIM', 'IMSI', 'IP', 'IMSI_MATCHED_METER_NO',
       'UDC_ID', 'IMSI_MATCHED_IP', 'INSTALLATIONNUMBER',
       'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY'],
      dtype='object')

In [59]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN]

In [60]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,INSTALLATIONNUMBER,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL


In [61]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"IMSI_MATCHED_METER_NO","IMSI_MATCHED_IP","IMSI_MATCHED_STATUS","IMSI_MATCHED_UTILITY"]]

### WORK ON LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED  and  LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED

In [62]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,INSTALLATIONNUMBER,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL
2,3,LT1027418,1 Phase,NBPDCL,S2S repaired,L&T,861996077250907,8991856040706794444,405856707339447,2405:0203:0530:EE5B:0000:0000:0000:0010,LT1027418,405856707339447,2405:0203:0530:EE5B:0000:0000:0000:0010,5001586266,Retired,NBPDCL


In [63]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL
2,3,LT1027418,1 Phase,NBPDCL,S2S repaired,L&T,861996077250907,8991856040706794444,405856707339447,2405:0203:0530:EE5B:0000:0000:0000:0010,LT1027418,405856707339447,2405:0203:0530:EE5B:0000:0000:0000:0010,5001586266,Retired,NBPDCL


In [64]:
#

# data correction

In [65]:
# Convert all columns to string data type

# change into str
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()

# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)




# Merge IMSI ,IP  and METER  file

In [66]:
warehouse_meter_imsi_data= pd.DataFrame(columns=[warehouse_Data.columns])

In [67]:
warehouse_meter_imsi_data

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP


In [68]:
#  empty column add to the  warehouse_meter_imsi_data  which has already some empty column of warehouse data

In [69]:
warehouse_meter_imsi_data['METER_MATCH_UDC_ID'] = None
warehouse_meter_imsi_data['METER_MATCH_IP'] = None
warehouse_meter_imsi_data['METER_MATCH_STATUS'] = None
warehouse_meter_imsi_data['METER_MATCH_UTILITY'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_IP'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_STATUS'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_UTILITY'] = None
warehouse_meter_imsi_data['IP_MATCHED_METER_NO'] = None
warehouse_meter_imsi_data['IP_MATCH_UDC_ID'] = None
warehouse_meter_imsi_data['IP_MATCHED_IP'] = None
warehouse_meter_imsi_data['IP_MATCHED_STATUS'] = None
warehouse_meter_imsi_data['IP_MATCHED_UTILITY'] = None


In [70]:
warehouse_meter_imsi_data

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY


In [71]:
# this column are not same in all the dataframe so that i used the above method

In [72]:
"""
warehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',
       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',
       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',
       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',
       'IMSI_MATCHED_UTILITY'])
"""

"\nwarehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',\n       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',\n       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',\n       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',\n       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',\n       'IMSI_MATCHED_UTILITY'])\n"

In [73]:
LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,INSTALLATIONNUMBER,IP_MATCHED_STATUS,IP_MATCHED_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL


In [74]:
warehouse_meter_imsi_data.head()

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY


In [75]:
# combining meter  and imsi data

#   METER AND IMSI MERGE

warehouse_meter_imsi_data_merge = pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED, LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[['METER NO.', 'IMSI','IMSI_MATCHED_METER_NO','IMSI_MATCHED_IP','IMSI_MATCHED_STATUS','IMSI_MATCHED_UTILITY']], left_on=['METER NO.', 'IMSI'], right_on=['METER NO.', 'IMSI'], how='left')


# MERGE IP AND warehouse_meter_imsi_data_merge
warehouse_meter_imsi_data = pd.merge(warehouse_meter_imsi_data_merge, LEFT_JOIN_WITH_IP_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[['METER NO.', 'IMSI','IP_MATCHED_METER_NO','IP_MATCH_UDC_ID','IP_MATCHED_IP','IP_MATCHED_STATUS','IP_MATCHED_UTILITY']], left_on=['METER NO.', 'IMSI'], right_on=['METER NO.', 'IMSI'], how='left')



In [76]:
warehouse_meter_imsi_data.head(2)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL,LT0364314,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL


In [77]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data = warehouse_meter_imsi_data.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data.columns = warehouse_meter_imsi_data.columns.str.upper()

# ----------------------------------------------------------------------------------


# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data = warehouse_meter_imsi_data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



In [78]:
warehouse_meter_imsi_data.shape

(1183, 25)

# SIM SET CHECK STATUS

In [79]:
def fun_sim_status_check(row):
    if (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "SAME SET IN SAME UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN SAME UTILITY"
    elif (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "SAME SET IN DIFFERENT UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    else:
        return None

# Apply the function to create a new column
warehouse_meter_imsi_data['SIM SET CHECK STATUS'] = warehouse_meter_imsi_data.apply(fun_sim_status_check, axis=1)



# TRUE - FALSE COLUMN

In [80]:
def fun_check_true_false__METER_SIM(row):
    if row['IMSI']==row['METER_MATCH_UDC_ID']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_SIM"] =  warehouse_meter_imsi_data.apply(fun_check_true_false__METER_SIM, axis=1)

In [81]:
def fun_check_true_false_METER_ORG(row):
    if row['METER_MATCH_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_ORG, axis=1)

In [82]:
def fun_check_true_false_SIM_ORG(row):
    if row['IMSI_MATCHED_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["SIM_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_SIM_ORG, axis=1)

In [83]:
def fun_check_true_false_METER_IP(row):
    if row['METER_MATCH_IP']==row['IP']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_IP"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_IP, axis=1)

In [84]:
warehouse_meter_imsi_data.head(2)


,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN SAME UTILITY,True,True,True,True
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL,LT0364314,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True


# SIM SET CHECK STATUS RECHECK COLUMN

In [85]:
def check_set_and_utility(row):
    if row['METER_SIM'] == True and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SAME SET IN SAME UTILITY"
    elif row['METER_SIM'] == True and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "SAME SET IN DIFFERENT UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "DIFFERENT SET IN SAME UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    
    # new case added
    elif row['METER_SIM'] == True and (row['METER_ORG'] == True) and (row["SIM_ORG"] == False) :
        return "SAME SET IN SAME UTILITY"
    elif row['METER_SIM'] == True and (row['METER_ORG'] == False) and (row["SIM_ORG"] == True) :
        return "SAME SET IN DIFFERENT UTILITY"
    
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == False)) :
        return "DIFFERENT SET IN SAME UTILITY"
    
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == True)) :
        return "DIFFERENT SET IN DIFFERENT UTILITY"

    
    else:
        return None


warehouse_meter_imsi_data["SIM SET CHECK STATUS RECHECK"] =  warehouse_meter_imsi_data.apply(check_set_and_utility, axis=1)


In [86]:
warehouse_meter_imsi_data.columns

Index(['SL NO', 'METER NO.', 'PHASE', 'WAREHOUSE_UTILITY', 'WAREHOUSE_STATUS',
       'MAKE', 'IME', 'FINAL SIM', 'IMSI', 'IP', 'MFG_SERIAL_NUM',
       'METER_MATCH_UDC_ID', 'METER_MATCH_IP', 'INSTALLATIONNUMBER',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY', 'IMSI_MATCHED_METER_NO',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY',
       'IP_MATCHED_METER_NO', 'IP_MATCH_UDC_ID', 'IP_MATCHED_IP',
       'IP_MATCHED_STATUS', 'IP_MATCHED_UTILITY', 'SIM SET CHECK STATUS',
       'METER_SIM', 'METER_ORG', 'SIM_ORG', 'METER_IP',
       'SIM SET CHECK STATUS RECHECK'],
      dtype='object')

# Action taken column

In [87]:

def fun_action_taken_case_I(data_row):

    # case 1
    
    #if data_row['IMSI']== data_row['METER_MATCH_UDC_ID']:
        #return "SET IS IN INSTALLED STAGE"
    if (data_row['METER_MATCH_STATUS'] != "Installed")  & (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"
    elif (data_row['METER_MATCH_STATUS'] == "Installed")  & (data_row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    

    # new cases
    
    # 1
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'SAME SET IN SAME UTILITY') and (data_row['METER_MATCH_STATUS'] == "Installed") and (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "SET IS IN INSTALLED STAGE"
    
    # 2
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'SAME SET IN DIFFERENT UTILITY') and (data_row['METER_MATCH_STATUS'] == "Installed") and (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "SET IS IN INSTALLED STAGE"
    
    # 3
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'DIFFERENT SET IN SAME UTILITY') and (data_row['METER_MATCH_STATUS'] == "Installed") and (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "BOTH INSTALLED IN SAME UTILITY"
    
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'DIFFERENT SET IN SAME UTILITY') and (data_row['METER_MATCH_STATUS'] == "Installed") and (data_row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'DIFFERENT SET IN SAME UTILITY') and (data_row['METER_MATCH_STATUS'] != "Installed") and (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"
    
    # 4
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'DIFFERENT SET IN DIFFERENT UTILITY') and (data_row['METER_MATCH_STATUS'] == "Installed") and (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "BOTH INSTALLED IN SAME UTILITY"
    
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'DIFFERENT SET IN DIFFERENT UTILITY') and (data_row['METER_MATCH_STATUS'] == "Installed") and (data_row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row["SIM SET CHECK STATUS RECHECK"] == 'DIFFERENT SET IN DIFFERENT UTILITY') and (data_row['METER_MATCH_STATUS'] != "Installed") and (data_row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"

    # case 2 


    # set 1 and row['METER_IP'] == True
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True)) and (data_row['METER_IP'] == True) and data_row['METER_MATCH_STATUS']!='Installed' and data_row['IMSI_MATCHED_STATUS'] !='Installed':
        return "METER-SIM LINK FILE REQUIRED"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True) and (data_row['METER_IP'] == True) and data_row['METER_MATCH_STATUS']=='Installed' :
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True) and (data_row['METER_IP'] == True) and  data_row['IMSI_MATCHED_STATUS']=='Installed' :
        return "IMSI IS IN INSTALLED STAGE"
    
    
    
    



    # set 1 and row['METER_IP'] == False 
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == True)) and (data_row['METER_IP'] == False) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == False)) and (data_row['METER_IP'] == False) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    # set 2  "SAME SET IN DIFFERENT UTILITY"
    elif (data_row['METER_SIM'] == True) and ((data_row['METER_ORG'] == False) and (data_row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"

    #set 3  "DIFFERENT SET IN SAME UTILITY"
    
    elif (data_row['METER_SIM'] == False) and ((data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == True)) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
 
    # set 4  "DIFFERENT SET IN DIFFERENT UTILITY"
    elif (data_row['METER_SIM'] == False) and ((data_row['METER_ORG'] == False) and (data_row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    
    elif (data_row['METER_SIM'] == False) and (data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False):
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    elif (data_row['METER_SIM'] == False) and (data_row['METER_ORG'] == True) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == True):
        return "SIM AND METER-SIM LINK FILE REQUIRED"


    # new cases are added in this to handle imsi null
    
    elif ( data_row['IMSI_MATCHED_METER_NO'] =='nan') and ( data_row['IMSI_MATCHED_IP'] =='nan') and ( data_row['IMSI_MATCHED_STATUS'] =='nan') and (data_row['METER_ORG'] == True ):
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( data_row['IMSI_MATCHED_METER_NO'] =='nan' ) and ( data_row['IMSI_MATCHED_IP'] =='nan') and ( data_row['IMSI_MATCHED_STATUS'] =='nan') and (data_row['METER_ORG'] == False ):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( data_row['IMSI_MATCHED_METER_NO'] =='nan') and ( data_row['IMSI_MATCHED_IP'] =='nan') and ( data_row['IMSI_MATCHED_STATUS'] =='nan')  and ( data_row['METER_MATCH_UDC_ID'] =='nan') and ( data_row['METER_MATCH_IP'] =='nan') and ( data_row['METER_MATCH_STATUS'] =='nan'):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    
    # set is in installed
    elif (data_row['METER NO.']+data_row['IMSI']) == data_row['METER NO.']+data_row['METER_MATCH_UDC_ID']:
        return "SET IS IN INSTALLED STAGE"
    
    

    
    else:
        return " NEED TO CHECK THIS FILE"
    
    
    
warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_case_I, axis=1)

In [88]:
"""
# new cases
    elif (data_row['METER_MATCH_UTILITY'] == warehouse_utility_importation_demand)  and  (data_row['METER_MATCH_STATUS']=='Installed') and (data_row['IMSI_MATCHED_UTILITY']== warehouse_utility_importation_demand) and (data_row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'BOTH INSTALLED IN SAME UTILITY'
    
    elif (data_row['METER_SIM'] == False) and (data_row['METER_ORG']  == False) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False) and data_row['METER_MATCH_STATUS']=='Installed' :
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == False) and (data_row['METER_ORG']  == False) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False) and  data_row['IMSI_MATCHED_STATUS']=='Installed' :
        return "IMSI IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == False) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False) and data_row['METER_MATCH_STATUS']=='Installed' :
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == False) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False) and  data_row['IMSI_MATCHED_STATUS']=='Installed' :
        return "IMSI IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False) and data_row['METER_MATCH_STATUS']=='Installed' :
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == False) and (data_row['METER_IP'] == False) and  data_row['IMSI_MATCHED_STATUS']=='Installed' :
        return "IMSI IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True) and (data_row['METER_IP'] == False) and data_row['METER_MATCH_STATUS']=='Installed' :
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == True) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True) and (data_row['METER_IP'] == False) and  data_row['IMSI_MATCHED_STATUS']=='Installed' :
        return "IMSI IS IN INSTALLED STAGE"

    elif (data_row['METER_SIM'] == False) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True) and (data_row['METER_IP'] == True) and data_row['METER_MATCH_STATUS']=='Installed' :
        return "METER IS IN INSTALLED STAGE"
    
    elif (data_row['METER_SIM'] == False) and (data_row['METER_ORG']  == True) and (data_row["SIM_ORG"] == True) and (data_row['METER_IP'] == True) and  data_row['IMSI_MATCHED_STATUS']=='Installed' :
        return "IMSI IS IN INSTALLED STAGE"
"""

'\n# new cases\n    elif (data_row[\'METER_MATCH_UTILITY\'] == warehouse_utility_importation_demand)  and  (data_row[\'METER_MATCH_STATUS\']==\'Installed\') and (data_row[\'IMSI_MATCHED_UTILITY\']== warehouse_utility_importation_demand) and (data_row[\'IMSI_MATCHED_STATUS\'] == \'Installed\'):\n        return \'BOTH INSTALLED IN SAME UTILITY\'\n    \n    elif (data_row[\'METER_SIM\'] == False) and (data_row[\'METER_ORG\']  == False) and (data_row["SIM_ORG"] == False) and (data_row[\'METER_IP\'] == False) and data_row[\'METER_MATCH_STATUS\']==\'Installed\' :\n        return "METER IS IN INSTALLED STAGE"\n    \n    elif (data_row[\'METER_SIM\'] == False) and (data_row[\'METER_ORG\']  == False) and (data_row["SIM_ORG"] == False) and (data_row[\'METER_IP\'] == False) and  data_row[\'IMSI_MATCHED_STATUS\']==\'Installed\' :\n        return "IMSI IS IN INSTALLED STAGE"\n    \n    elif (data_row[\'METER_SIM\'] == True) and (data_row[\'METER_ORG\']  == False) and (data_row["SIM_ORG"] == False) 

In [89]:
warehouse_meter_imsi_data.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_meter_imsi_data.csv",index=False)

In [90]:
warehouse_meter_imsi_data.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED


# user option

In [91]:
warehouse_meter_imsi_data['ACTION TAKEN'].unique()

array(['METER-SIM LINK FILE REQUIRED',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'SET IS IN INSTALLED STAGE', 'IMSI IS IN INSTALLED STAGE',
       'METER IS IN INSTALLED STAGE', 'BOTH INSTALLED IN SAME UTILITY',
       ' NEED TO CHECK THIS FILE'], dtype=object)

In [92]:
# SEPRATE IMSI,METER,SET INSTALLED  AND OTHER ACTION TAKEN

warehouse_meter_imsi_set_installed_Stage = warehouse_meter_imsi_data[(warehouse_meter_imsi_data['ACTION TAKEN'] == "METER IS IN INSTALLED STAGE") | 
                           (warehouse_meter_imsi_data['ACTION TAKEN'] == "IMSI IS IN INSTALLED STAGE") |
                           (warehouse_meter_imsi_data['ACTION TAKEN'] == "SET IS IN INSTALLED STAGE") | 
                           (warehouse_meter_imsi_data['ACTION TAKEN'] == "BOTH INSTALLED IN SAME UTILITY")]

warehouse_meter_imsi_other_Action_taken = warehouse_meter_imsi_data[(warehouse_meter_imsi_data['ACTION TAKEN']== "SIM AND METER-SIM LINK FILE REQUIRED") |
                                                                    (warehouse_meter_imsi_data['ACTION TAKEN']=="METER-SIM LINK FILE REQUIRED") |
                                                                    (warehouse_meter_imsi_data['ACTION TAKEN']=="METER , SIM AND METER-SIM LINK FILE REQUIRED")]

In [93]:
# FOR NBPDCL

def fun_meter_sim_set_installed_Stage(row):
    if (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS']=='Installed') and (row['IMSI_MATCHED_UTILITY']== 'NBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'BOTH INSTALLED IN SAME UTILITY'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS']=='Installed') and (row['IMSI_MATCHED_UTILITY']== 'NBPDCL') and (row['IMSI_MATCHED_STATUS'] != 'Installed'):
        return 'METER IS IN INSTALLED STAGE'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'NBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'IMSI IS IN INSTALLED STAGE'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] =='Installed') and (row['IMSI_MATCHED_UTILITY']== 'nan') and (row['IMSI_MATCHED_STATUS'] == 'nan'):
        return 'METER IS IN INSTALLED STAGE'
    
    #  another cases
    
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'SIM AND METER-SIM LINK FILE REQUIRED'
    elif (row['METER_MATCH_UTILITY'] == 'NBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS']=='nan'):
        return 'SIM AND METER-SIM LINK FILE REQUIRED'
    
    elif (row['METER_MATCH_UTILITY'] == 'SBPDCL')  and  (row['METER_MATCH_STATUS'] =='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'METER , SIM AND METER-SIM LINK FILE REQUIRED'
    elif (row['METER_MATCH_UTILITY'] == 'SBPDCL')  and  (row['METER_MATCH_STATUS'] !='Installed') and (row['IMSI_MATCHED_UTILITY']== 'SBPDCL') and (row['IMSI_MATCHED_STATUS'] == 'Installed'):
        return 'METER , SIM AND METER-SIM LINK FILE REQUIRED'
  
    

In [94]:

if warehouse_utility_importation_demand == "1":
      
    warehouse_meter_imsi_set_installed_Stage.drop(['ACTION TAKEN'], axis=1)    
    warehouse_meter_imsi_set_installed_Stage['ACTION TAKEN'] =  warehouse_meter_imsi_set_installed_Stage.apply(fun_meter_sim_set_installed_Stage, axis=1)

    warehouse_meter_imsi_set_installed_Stage.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_meter_imsi_set_installed_Stage.csv",index=False)

    #concat
    warehouse_meter_imsi_data=pd.concat([warehouse_meter_imsi_other_Action_taken, warehouse_meter_imsi_set_installed_Stage], ignore_index=True)
    warehouse_meter_imsi_data.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_meter_imsi_data.csv",index=False)

    
elif warehouse_utility_importation_demand == "2":
    print(" working on SBPDCL data")
elif warehouse_utility_importation_demand == "3":
    print("meter is new , new imsi - code flow is same as before no interuption")
else:
    print("plz chose correct option")


meter is new , new imsi - code flow is same as before no interuption


In [95]:
warehouse_meter_imsi_set_installed_Stage.columns

Index(['SL NO', 'METER NO.', 'PHASE', 'WAREHOUSE_UTILITY', 'WAREHOUSE_STATUS',
       'MAKE', 'IME', 'FINAL SIM', 'IMSI', 'IP', 'MFG_SERIAL_NUM',
       'METER_MATCH_UDC_ID', 'METER_MATCH_IP', 'INSTALLATIONNUMBER',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY', 'IMSI_MATCHED_METER_NO',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY',
       'IP_MATCHED_METER_NO', 'IP_MATCH_UDC_ID', 'IP_MATCHED_IP',
       'IP_MATCHED_STATUS', 'IP_MATCHED_UTILITY', 'SIM SET CHECK STATUS',
       'METER_SIM', 'METER_ORG', 'SIM_ORG', 'METER_IP',
       'SIM SET CHECK STATUS RECHECK', 'ACTION TAKEN'],
      dtype='object')

In [96]:
warehouse_meter_imsi_data[warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO']!='nan'].count()

SL NO                           1183
METER NO.                       1183
PHASE                           1183
WAREHOUSE_UTILITY               1183
WAREHOUSE_STATUS                1183
MAKE                            1183
IME                             1183
FINAL SIM                       1183
IMSI                            1183
IP                              1183
MFG_SERIAL_NUM                  1183
METER_MATCH_UDC_ID              1183
METER_MATCH_IP                  1183
INSTALLATIONNUMBER              1183
METER_MATCH_STATUS              1183
METER_MATCH_UTILITY             1183
IMSI_MATCHED_METER_NO           1183
IMSI_MATCHED_IP                 1183
IMSI_MATCHED_STATUS             1183
IMSI_MATCHED_UTILITY            1183
IP_MATCHED_METER_NO             1183
IP_MATCH_UDC_ID                 1183
IP_MATCHED_IP                   1183
IP_MATCHED_STATUS               1183
IP_MATCHED_UTILITY              1183
SIM SET CHECK STATUS            1183
METER_SIM                       1183
M

In [97]:
warehouse_meter_imsi_data.head(2)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL,LT0364314,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED


# Finally done

In [98]:
warehouse_meter_imsi_data.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED


In [99]:

# Custom function to update meter numbers
def update_meter_number(meter_number):
    if meter_number.startswith('GP') or meter_number.startswith('GL'):
        return 'GOE' + meter_number
    elif meter_number.startswith('LT'):
        return 'LNT' + meter_number
    else:
        return meter_number

# Apply the custom function to each element in the 'METER NO.' column
warehouse_meter_imsi_data['NEW_METER_NUMBER'] = warehouse_meter_imsi_data['METER NO.'].apply(update_meter_number)




In [100]:
# adding duplicate and unique in data set 

warehouse_meter_imsi_data['Duplicated'] = warehouse_meter_imsi_data.duplicated(subset=['METER NO.']).map({True: 'Duplicated', False: 'Unique'})

# FINAL FILE SAVE AT MULTIPLE LOCATION

In [101]:
# Final file at desktop folder

warehouse_meter_imsi_data.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_Data_meter_sim_final.csv',index=False)


# Final file in code folder

warehouse_meter_imsi_data.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv",index=False)

In [102]:
warehouse_meter_imsi_data_check=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv")

In [103]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()

# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

# NEED TO CHECK FILE

In [104]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM = warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']==' NEED TO CHECK THIS FILE']

In [105]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv",index=False)
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv',index=False)



In [106]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV = pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv")

In [107]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV.head()

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,DUPLICATED


In [108]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'].isnull()]

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,DUPLICATED


In [109]:
warehouse_meter_imsi_data_check.head(2)


,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,DUPLICATED
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561.0,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED,LNTLT0164550,Unique
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335.0,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL,LT0364314,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED,LNTLT0364314,Unique


In [110]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'] != "NEED TO CHECK THIS FILE"].count()

SL NO                           1181
METER NO.                       1181
PHASE                           1181
WAREHOUSE_UTILITY               1181
WAREHOUSE_STATUS                1181
MAKE                            1181
IME                             1181
FINAL SIM                       1181
IMSI                            1181
IP                              1181
MFG_SERIAL_NUM                  1181
METER_MATCH_UDC_ID              1181
METER_MATCH_IP                  1181
INSTALLATIONNUMBER              1181
METER_MATCH_STATUS              1181
METER_MATCH_UTILITY             1181
IMSI_MATCHED_METER_NO           1181
IMSI_MATCHED_IP                 1181
IMSI_MATCHED_STATUS             1181
IMSI_MATCHED_UTILITY            1181
IP_MATCHED_METER_NO             1181
IP_MATCH_UDC_ID                 1181
IP_MATCHED_IP                   1181
IP_MATCHED_STATUS               1181
IP_MATCHED_UTILITY              1181
SIM SET CHECK STATUS            1181
METER_SIM                       1181
M

In [111]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER-SIM LINK FILE REQUIRED',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'SET IS IN INSTALLED STAGE', 'IMSI IS IN INSTALLED STAGE',
       'METER IS IN INSTALLED STAGE', 'BOTH INSTALLED IN SAME UTILITY',
       'NEED TO CHECK THIS FILE'], dtype=object)

In [112]:
warehouse_meter_imsi_data_check.head(2)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,DUPLICATED
0,1,LT0164550,1 Phase,SBPDCL,S2S repaired,L&T,861190055985524,8991856050706095561,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,LT0164550,405856706095561.0,2405:0203:0522:9F13:0000:0000:0000:0010,5002548022,Retired,SBPDCL,LT0164550,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,LT0164550,405856706095561,2405:0203:0522:9F13:0000:0000:0000:0010,Retired,SBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED,LNTLT0164550,Unique
1,2,LT0364314,1 Phase,NBPDCL,S2S repaired,L&T,861996077248679,8991856050706402335,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,LT0364314,405856706402335.0,2405:0203:0525:C6A1:0000:0000:0000:0010,5002884613,Retired,NBPDCL,LT0364314,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,LT0364314,405856706402335,2405:0203:0525:C6A1:0000:0000:0000:0010,Retired,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,METER-SIM LINK FILE REQUIRED,LNTLT0364314,Unique


In [113]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER-SIM LINK FILE REQUIRED',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'SET IS IN INSTALLED STAGE', 'IMSI IS IN INSTALLED STAGE',
       'METER IS IN INSTALLED STAGE', 'BOTH INSTALLED IN SAME UTILITY',
       'NEED TO CHECK THIS FILE'], dtype=object)

In [114]:
warehouse_meter_imsi_data_check['SIM SET CHECK STATUS RECHECK'].unique()

array(['SAME SET IN SAME UTILITY', 'DIFFERENT SET IN SAME UTILITY',
       'DIFFERENT SET IN DIFFERENT UTILITY'], dtype=object)

In [115]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['IMSI_MATCHED_METER_NO']=="nan"].count()

SL NO                           0
METER NO.                       0
PHASE                           0
WAREHOUSE_UTILITY               0
WAREHOUSE_STATUS                0
MAKE                            0
IME                             0
FINAL SIM                       0
IMSI                            0
IP                              0
MFG_SERIAL_NUM                  0
METER_MATCH_UDC_ID              0
METER_MATCH_IP                  0
INSTALLATIONNUMBER              0
METER_MATCH_STATUS              0
METER_MATCH_UTILITY             0
IMSI_MATCHED_METER_NO           0
IMSI_MATCHED_IP                 0
IMSI_MATCHED_STATUS             0
IMSI_MATCHED_UTILITY            0
IP_MATCHED_METER_NO             0
IP_MATCH_UDC_ID                 0
IP_MATCHED_IP                   0
IP_MATCHED_STATUS               0
IP_MATCHED_UTILITY              0
SIM SET CHECK STATUS            0
METER_SIM                       0
METER_ORG                       0
SIM_ORG                         0
METER_IP      

In [116]:
warehouse_meter_imsi_data_check['METER_MATCH_UDC_ID'].dtype

dtype('O')

In [117]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER-SIM LINK FILE REQUIRED',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'SET IS IN INSTALLED STAGE', 'IMSI IS IN INSTALLED STAGE',
       'METER IS IN INSTALLED STAGE', 'BOTH INSTALLED IN SAME UTILITY',
       'NEED TO CHECK THIS FILE'], dtype=object)

In [118]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE'].count()

SL NO                           2
METER NO.                       2
PHASE                           2
WAREHOUSE_UTILITY               2
WAREHOUSE_STATUS                2
MAKE                            2
IME                             2
FINAL SIM                       2
IMSI                            2
IP                              2
MFG_SERIAL_NUM                  2
METER_MATCH_UDC_ID              2
METER_MATCH_IP                  2
INSTALLATIONNUMBER              2
METER_MATCH_STATUS              2
METER_MATCH_UTILITY             2
IMSI_MATCHED_METER_NO           2
IMSI_MATCHED_IP                 2
IMSI_MATCHED_STATUS             2
IMSI_MATCHED_UTILITY            2
IP_MATCHED_METER_NO             2
IP_MATCH_UDC_ID                 2
IP_MATCHED_IP                   2
IP_MATCHED_STATUS               2
IP_MATCHED_UTILITY              2
SIM SET CHECK STATUS            2
METER_SIM                       2
METER_ORG                       2
SIM_ORG                         2
METER_IP      

In [119]:
#assest_initiation_invent_sb[assest_initiation_invent_sb['UDC_ID']=='404751250008001']

In [120]:
#assest_initiation_invent_nb[assest_initiation_invent_nb['UDC_ID']=='404751250008001']

In [121]:
warehouse_meter_imsi_data_check['DUPLICATED'].unique()

array(['Unique', 'Duplicated'], dtype=object)

In [122]:
warehouse_meter_imsi_data_check.shape

(1183, 34)

# extra


####  THIS WILL REMOVE THE DUPLICATE  

In [123]:
#warehouse_meter_imsi_data_check.drop[warehouse_meter_imsi_data_check['DUPLICATED']=='Duplicated']

# Assuming warehouse_meter_imsi_data_check is your DataFrame


#warehouse_meter_imsi_data_check.drop(warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['DUPLICATED'] == 'Duplicated'].index, inplace=True)


In [124]:
warehouse_meter_imsi_data_check.shape

(1183, 34)

# creating meter-sim link file

In [125]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)
# ----------------------------------------------------------------------------------
# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()
# ----------------------------------------------------------------------------------
# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



In [126]:
#METER , SIM AND METER-SIM LINK FILE REQUIRED

#SIM AND METER-SIM LINK FILE REQUIRED

#METER-SIM LINK FILE REQUIRED
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['METER-SIM LINK FILE REQUIRED',
       'SIM AND METER-SIM LINK FILE REQUIRED',
       'SET IS IN INSTALLED STAGE', 'IMSI IS IN INSTALLED STAGE',
       'METER IS IN INSTALLED STAGE', 'BOTH INSTALLED IN SAME UTILITY',
       'NEED TO CHECK THIS FILE'], dtype=object)

In [127]:
# creating meter_sim_link_file

#CREATE_METER_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER,SIM AND METER-SIM LINK FILE REQUIRED') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER IS IN INSTALLED STAGE') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER , SIM AND METER-SIM LINK FILE REQUIRED')]

CREATE_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='SIM AND METER-SIM LINK FILE REQUIRED')]

CREATE_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER-SIM LINK FILE REQUIRED')]
                                                                          

In [128]:
print(f" CREATE_METER_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")
print(f" CREATE_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : {CREATE_SIM_AND_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")
print(f" CREATE_METER_SIM_LINK_FILE - UNIQUE : {CREATE_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")


 CREATE_METER_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : []
 CREATE_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : ['SIM AND METER-SIM LINK FILE REQUIRED']
 CREATE_METER_SIM_LINK_FILE - UNIQUE : ['METER-SIM LINK FILE REQUIRED']


### Meter-sim link file for all 3 cases

In [129]:
# this file is meter-sim link code

In [130]:
def fun_meter_sim_link_file(row):

    return "{};EP2PMeter;{};{};{}".format(row['NEW_METER_NUMBER'], row['IMSI'],row['FINAL SIM'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data'] = CREATE_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_23448\2655918992.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_23448\2655918992.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data'] = CREATE_METER_SIM_LINK_FILE.apply(fun_met

In [131]:
#EESL_BSNL_PO0000000402_00000_SIM_20231220_121002_SBPDCL
#404751250004001;8991759065212004001;fd00:0000:0800:CC19:0000:0000:0000:0001;;BSNLEDF;ACTIVE;SBPDCL

#  this code is for sim file for all the 2 cases

In [132]:
def fun_sim_file(row):

    return "{};{};{};;BSNLEDF;ACTIVE;{}".format(row['IMSI'], row['FINAL SIM'],row['IP'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)
CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)


C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_23448\669809455.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)


# THIS CODE IS FOR METER FILE FOR METER,SIM,METER-SIM LINK FILE

In [133]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE'].unique()

array([], dtype=object)

##### 1 phase and 3 phase

In [134]:
# 1 phase and 3 phase meter file :   meter, sim , meter-sim link file 


CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE']=='1 Phase']
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE['NEW_METER_NUMBER'].str.startswith('GOE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE['NEW_METER_NUMBER'].str.startswith('LNT')]



CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE']=='3 Phase']
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE['NEW_METER_NUMBER'].str.startswith('GOE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE['NEW_METER_NUMBER'].str.startswith('LNT')]


In [135]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.head(1)

,SL NO,METER NO.,PHASE,WAREHOUSE_UTILITY,WAREHOUSE_STATUS,MAKE,IME,FINAL SIM,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,INSTALLATIONNUMBER,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,IP_MATCHED_METER_NO,IP_MATCH_UDC_ID,IP_MATCHED_IP,IP_MATCHED_STATUS,IP_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,DUPLICATED,"meter_sim_link_(M,S & M-S)_single_file_data","sim_link_(M,S & M-S)_single_file_data"


### 1 PHASE

In [136]:
def fun_meter_1_PHASE_GENUS_file(row):

    return "{};{};EP2PMeter;6_D1;SKM145-s2;GOE;2019;2019;DLMS;;;G36B7.145090;;;3G;;;;;{};4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_1PH_WC;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS.apply(fun_meter_1_PHASE_GENUS_file, axis=1)

In [137]:
def fun_meter_1_PHASE_LNT_file(row):

    return "{};{};EP2PMeter;6_D1;AURORA;LNT;2021;2021;DLMS;;;RN13.00 00;;8;3G;;;;;{};4059;TCP;32;;;;lnt1;2;1;0;EESL_1PH_WC_V1;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT.apply(fun_meter_1_PHASE_LNT_file, axis=1)

### 3 PHASE

In [138]:
def fun_meter_3_PHASE_GENUS_file(row):

    return "{};{};EP2PMeter;7_D2;SKM345-s2;GOE;2019;2019;DLMS;;;GF6B7.325061;;;3G;;;;;{};4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_3PH_WC;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS.apply(fun_meter_3_PHASE_GENUS_file, axis=1)

In [139]:
def fun_meter_3_PHASE_LNT_file(row):

    return "{};{};EP2PMeter;7_D2;REGOR;LNT;2021;2021;DLMS;;;3PBNC.82 01;;8;3G;;;;;{};4059;TCP;32;;;;lnt1;2;1;0;EESL_3PH_WC_V1;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT.apply(fun_meter_3_PHASE_LNT_file, axis=1)

#### concat file

In [140]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL = pd.concat([CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS, CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT,CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS,CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT], ignore_index=True)

In [141]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL.csv",index=False)
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)

In [142]:
# assign :  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL  - to -   CREATE_METER_SIM_AND_METER_SIM_LINK_FILE

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL

In [143]:
# SAVING METER-SIM LINK FILE TO OUTPUT LOCATION

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)
CREATE_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)
CREATE_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_METER_SIM_LINK_FILE.csv",index=False)

# CREATING A NEW DATA FRAME

#### WORKING ON METER-SIM LINK FILE 

In [144]:
# 1 
# sim-meter link in meter,sim and meter-sim link file

meter_sim_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data']
if  not (meter_sim_link_M_S_AND_MS_LINK_file_df.empty):
    meter_sim_link_M_S_AND_MS_LINK_file_df.to_csv("S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK.head(1)
else:
    print(" meter_sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


 meter_sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME


In [145]:
# 2
# sim-meter link in sim and meter-sim link file

meter_sim_link_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_S_AND_MS_LINK_file_df=CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data']
if  not (meter_sim_link_S_AND_MS_LINK_file_df.empty):
    meter_sim_link_S_AND_MS_LINK_file_df.to_csv("S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_S_AND_MS_LINK.head(1)
else:
    print(" meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [146]:
# 3
# sim-meter link in meter-sim link file

meter_sim_link_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_MS_LINK_file_df=CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data']
if  not (meter_sim_link_MS_LINK_file_df.empty):
    meter_sim_link_MS_LINK_file_df.to_csv("M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_MS_LINK.head(1)
else:
    print(" meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


### working on sim link file

In [147]:
# 1
# sim link file in meter,sim and meter-sim link file

sim_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
sim_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] 
if  not (sim_link_M_S_AND_MS_LINK_file_df.empty):
    sim_link_M_S_AND_MS_LINK_file_df.to_csv("M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None,index=False)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_M_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_M_AND_MS_LINK.head(1)
else:
    print(" sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


 sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME


In [148]:
# 2 
# sim link file in sim and meter-sim link file

sim_link_S_AND_MS_LINK_file_df = pd.DataFrame()
sim_link_S_AND_MS_LINK_file_df=CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data']
if  not (sim_link_S_AND_MS_LINK_file_df.empty):
    sim_link_S_AND_MS_LINK_file_df.to_csv("S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None,index=False)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_AND_MS_LINK.head(1)
else:
    print(" sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")

### working on meter-link file

In [149]:
# 1 
# meter link in meter,sim and meter-sim link file

meter_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['Meter_link_(M,S & M-S)_single_file_data']
if  not (meter_link_M_S_AND_MS_LINK_file_df.empty):
    meter_link_M_S_AND_MS_LINK_file_df.to_csv("S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_EP2PMeter_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_EP2PMeter_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.head(1)
else:
    print(" meter_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")

 meter_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME


In [150]:
print(f"CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL: -   shape: - {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL.shape[0]}")
print(f"CREATE_METER_SIM_AND_METER_SIM_LINK_FILE: -   shape: - {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.shape[0]}")


CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_COCAT_ALL: -   shape: - 0
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE: -   shape: - 0


In [151]:
"""
new_df_check = EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.loc[2]
#print(f"len of new_df_check: {new_df_check}")
a=';'
word=""
for i in new_df_check:
    print(i)
    for j in i:
        word=word+j
        if a in j:
            #print(a)
            #print(f"word is : {word}")
            word=""
            #print("semicolon is here")
print(f"last word is : {word}")


"""

'\nnew_df_check = EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.loc[2]\n#print(f"len of new_df_check: {new_df_check}")\na=\';\'\nword=""\nfor i in new_df_check:\n    print(i)\n    for j in i:\n        word=word+j\n        if a in j:\n            #print(a)\n            #print(f"word is : {word}")\n            word=""\n            #print("semicolon is here")\nprint(f"last word is : {word}")\n\n\n'

### funciton for del and create the  folder on particular path


In [152]:
def folder_del_create(file_path):
    if os.path.exists(file_path):
        try:
            # Delete the folder and its contents
            shutil.rmtree(file_path)
            os.makedirs(file_path)
            print( "Folder at  deleted successfully.")
            print( "Folder at  created successfully.")
        except Exception as e:
            print(f"Error deleting folder: {e}")


#  METER,SIM AND METER-SIM LINK
M_S_AND_MS_MSLINK_FILE = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'
M_S_AND_MS_M_FILE = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER_FILE'
M_S_AND_MS_S_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\SIM_FILE'

#
S_AND_MSLINK_MSLINK_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE'
S_AND_MSLINK_S_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\SIM_FILE'

#METER-SIM LINK
MS_AND_MS_MSLINK_FILE=r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER-SIM LINK\METER-SIM_LINK_FILE'

folder_del_create(M_S_AND_MS_MSLINK_FILE)

Folder at  deleted successfully.
Folder at  created successfully.


In [153]:
folder_del_create(M_S_AND_MS_MSLINK_FILE)
folder_del_create(M_S_AND_MS_M_FILE)
folder_del_create(M_S_AND_MS_S_FILE)

folder_del_create(S_AND_MSLINK_MSLINK_FILE)
folder_del_create(S_AND_MSLINK_S_FILE)


folder_del_create(MS_AND_MS_MSLINK_FILE)

Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.


# function for generate single file

In [154]:
def generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix):
    data_frame_concat_data = pd.read_csv(df_file_path, header=None)

    # base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'

    num_folder = 0
    k = 100
    # new_folder_name = "EESL_GENUS_SINGLE_METER"

    new_folder_path = os.path.join(base_path, new_folder_name)
    os.makedirs(new_folder_path)

    # CURRENT_TIME_STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Iterate through each row and save it as a separate file
    for index, row in data_frame_concat_data.iterrows():
        
        # Create a new DataFrame with the current row
        single_row_df = pd.DataFrame([row])
        #print(f"single_row_data : {single_row_df}")
        last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
        #print(f"last_word : {last_part_after_semicolon}")


        CURRENT_TIME_STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
        #print(CURRENT_TIME_STAMP)
        
        #print(k)
        #print("-----------------")
        
        # Create the full path for the new folder

        if k > 995:
            k = 100
            final_new_folder_name = new_folder_name
            num_folder = num_folder + 1
            final_new_folder_name = new_folder_name + "_" + str(num_folder)
            new_folder_path = os.path.join(base_path, final_new_folder_name)
            os.makedirs(new_folder_path)
        else:
            file_name = generated_file_name_prefix + str(k) + generated_file_name_suffix + str(CURRENT_TIME_STAMP) + "_" + str(last_part_after_semicolon) + ".csv"
            #print(f"file name is {file_name}")
            file_path = os.path.join(new_folder_path, file_name)
            #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
            single_row_df.to_csv(file_path, index=False, header=False)
            k=k+1

        
    print("METER-SIM LINK FILE DONE")


In [155]:
# METER-SIM LINK

# 1. EESL_GENUS_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{utility}.csv


# SIM LINK FILE

# 2.  EESL_BSNL_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{utility}.csv
# 3.  EESL_GENUS_PO0000000{k}_00000_SIM_{CURRENT_TIME_STAMP}_{utility}.csv

# METER FILE

# 4. EESL_GENUS_PO0000000{K}_00000_EP2PMeter_{CURRENT_TIME_STAMP}_{utility}.csv

In [156]:

# SIM-Meter link file for all the three action taken...

# METER,SIM AND METER-SIM LINK  ----  METER-SIM_LINK_FILE
df_file_path =r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'


try:
    generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)
except Exception as e:
    # If an error occurs, print a custom message
    print("An error occurred while executing the function:")


An error occurred while executing the function:


In [157]:

#change the df file path
#  SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE


df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'


try:
    generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)
except Exception as e:
    # If an error occurs, print a custom message
    print("An error occurred while executing the function:")

METER-SIM LINK FILE DONE


In [158]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER-SIM LINK\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'


try:
    generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)
except Exception as e:
    # If an error occurs, print a custom message
    print("An error occurred while executing the function:")

METER-SIM LINK FILE DONE


In [159]:
# function for sim file 

In [160]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\SIM_FILE'
new_folder_name = 'EESL_BSNL_SINGLE_METER'
generated_file_name_prefix = 'EESL_BSNL_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'


try:
    generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)
except Exception as e:
    # If an error occurs, print a custom message
    print("An error occurred while executing the function:")

An error occurred while executing the function:


In [161]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\SIM_FILE'
new_folder_name = 'EESL_BSNL_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_'


try:
    generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)
except Exception as e:
    # If an error occurs, print a custom message
    print("An error occurred while executing the function:")

METER-SIM LINK FILE DONE


In [162]:
# meter file function


# METER,SIM AND METER-SIM LINK  ----  METER_LINK_FILE
df_file_path =r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_EP2PMeter_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_EP2PMeter_'



try:
    generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)
except Exception as e:
    # If an error occurs, print a custom message
    print("An error occurred while executing the function:")

An error occurred while executing the function:


In [163]:
#EESL_BSNL_PO0000000402_00000_SIM_20231220_121002_SBPDCL
#404751250004001;8991759065212004001;fd00:0000:0800:CC19:0000:0000:0000:0001;;BSNLEDF;ACTIVE;SBPDCL

In [164]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\METER_IMPORTATION_ZIMP_FILE.zip'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\METER_IMPORTATION_ZIMP_FILE.zip


In [165]:
  THE END LETS CHECK

SyntaxError: invalid syntax (4184175093.py, line 1)

: 

: 

# XML CODE

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime
def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")
for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    semicolon_split = rows[0].split(';')
    CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    if semicolon_split[3] == '6_D1':
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "1", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
        ]
    else:
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "3", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
        ]

    # Start building the XML string
    xml_string = '<SDPSyncMessage xmlns="http://www.emeter.com/energyip/syncinterface/v8" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n'

    # Loop through the data and append to the XML string
    for row in csv_data:
        
        if row.get("verb"):
            xml_string += '    <header>\n'
        elif row.get("mRID"):
            xml_string += '    <payload>\n        <device>\n'
        elif row.get("name"):
            xml_string += '            <parameter>\n'
        else:
            xml_string += '    <payload>\n        <device>\n'
        
        for key, value in row.items():
            if key != "verb" and key != "mRID" and key != "name":
                if key != "value" and key != "startDate":
                    xml_string += f'            <{key}>{value}</{key}>\n'
                else:
                    xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "value" and key == "":
                xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "verb":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "mRID":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "name":
                xml_string += f'                <{key}>{value}</{key}>\n'



        if row.get("verb"):
            xml_string += '    </header>\n'
        elif row.get("name"):
            xml_string += '            </parameter>\n'
        #else:
            #xml_string += '        </device>\n    </payload>\n'

    # Close the root element
    xml_string += '         </device>\n     </payload>\n</SDPSyncMessage>'

    # Print the XML string for verification
    print(xml_string)
    # Save the XML string to a file
    with open('output.xml', 'w') as file:
        file.write(xml_string)

    # Print a message indicating successful save
    print("XML string saved to 'output.xml'")


    # Parse the XML string
    root = ET.fromstring(xml_string)
    break
    
    
        

: 

: 

In [ ]:
# XML CODE WITH OUTPUT SAVE IN DIFF FILE
import pandas as pd
EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\New folder\Importation File\1PH\EESL_GENUS_PO0000000609_00000_EP2PMeter_20240405_075800_NBPDCL.csv")
EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.head()

,data
0,LNTLT0178071;LT0178071;EP2PMeter;6_D1;AURORA;L...
1,LNTLT0400754;LT0400754;EP2PMeter;6_D1;AURORA;L...
2,LNTLT0150608;LT0150608;EP2PMeter;6_D1;AURORA;L...
3,LNTLT0125960;LT0125960;EP2PMeter;6_D1;AURORA;L...


In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime
import os

def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")


CURRENT_TIME = datetime.now().strftime('%Y%m%d_%H%M%S')

num_folder = 0
k = 100
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE'
new_folder_name = 'METER_XML_OUTPUT_M_S_&_MS_LINK'


new_folder_path = os.path.join(base_path, new_folder_name)
shutil.rmtree(new_folder_path)
os.makedirs(new_folder_path)


for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    print(rows)
    semicolon_split_for_file_name = rows[0].split(';')
    semicolon_split = rows[0].split(';')
            
    CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    if semicolon_split[3] == '6_D1':
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "1", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
            ]
    else:
        csv_data = [
            {"verb": "SDPSync", "noun": "SDPSync", "revision": "1", "dateTime": CURRENT_TIME_STAMP, "source": "CIS", "messageID": "MESSAGE-10001", "asyncReplyTo": "none", "syncMode": "sync", "optimizationLevel": "Optimistic"},
            {"mRID": semicolon_split[0], "className": "EP2PMeter", "type": "Meter", "virtualInd": "N", "description": "Meter Description", "model": semicolon_split[4], "manufacturedDate": semicolon_split[6], "mfgSerialNumber": semicolon_split[1], "testDate": semicolon_split[6], "status": "Meter Shop", "commTechnology": "3G", "badgeId": semicolon_split[0], "sandardId": semicolon_split[0], "electronicId": semicolon_split[0], "make": semicolon_split[5], "deviceFunctionType": "N", "utility_id": semicolon_split[-1]},
            {"name": "Phase Type", "value": "3", "startDate": get_current_date()},
            {"name": "Device Protocol", "value": "DLMS", "startDate": get_current_date()},
            {"name": "Firmware Version", "value": semicolon_split[11], "startDate": get_current_date()},
            {"name": "DLMS Port", "value": "4059", "startDate": get_current_date()}
            ]
    xml_string = '<SDPSyncMessage xmlns="http://www.emeter.com/energyip/syncinterface/v8" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n'

                # Loop through the data and append to the XML string
    for row in csv_data:
            
        if row.get("verb"):
            xml_string += '    <header>\n'
        elif row.get("mRID"):
            xml_string += '    <payload>\n        <device>\n'
        elif row.get("name"):
            xml_string += '            <parameter>\n'
        else:
            xml_string += '    <payload>\n        <device>\n'

        for key, value in row.items():
            if key != "verb" and key != "mRID" and key != "name":
                if key != "value" and key != "startDate":
                    xml_string += f'            <{key}>{value}</{key}>\n'
                else:
                    xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "value" and key == "":
                xml_string += f'                <{key}>{value}</{key}>\n'
            elif key == "verb":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "mRID":
                xml_string += f'            <{key}>{value}</{key}>\n'
            elif key == "name":
                xml_string += f'                <{key}>{value}</{key}>\n'

        if row.get("verb"):
            xml_string += '    </header>\n'
        elif row.get("name"):
            xml_string += '            </parameter>\n'
            #else:
                #xml_string += '        </device>\n    </payload>\n'\
            
    xml_string += '         </device>\n     </payload>\n</SDPSyncMessage>'
            #root = ET.fromstring(xml_string)
    root = ET.ElementTree(ET.fromstring(xml_string))

    k=k+1 

    if k > 995:
            k = 100
            final_new_folder_name = new_folder_name
            num_folder = num_folder + 1
            final_new_folder_name = new_folder_name + "_" + str(num_folder)
            new_folder_path = os.path.join(base_path, final_new_folder_name)
            os.makedirs(new_folder_path)
    else:
        file_name = "Sync_Electric_Meter_" + semicolon_split_for_file_name[0] + "_"  + semicolon_split_for_file_name[-1] + "_" + str(CURRENT_TIME)  + "-" + str(k) + ".xml"
        print(file_name)
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        print(file_path)

        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        with open(file_path, 'w') as file:
            file.write(xml_string)
        
    
        

data    LNTLT0178071;LT0178071;EP2PMeter;6_D1;AURORA;L...
Name: 0, dtype: object
Sync_Electric_Meter_LNTLT0178071_NBPDCL_20240405_122504-101.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\METER_XML_OUTPUT_M_S_&_MS_LINK\Sync_Electric_Meter_LNTLT0178071_NBPDCL_20240405_122504-101.xml
data    LNTLT0400754;LT0400754;EP2PMeter;6_D1;AURORA;L...
Name: 1, dtype: object
Sync_Electric_Meter_LNTLT0400754_NBPDCL_20240405_122504-102.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\METER_XML_OUTPUT_M_S_&_MS_LINK\Sync_Electric_Meter_LNTLT0400754_NBPDCL_20240405_122504-102.xml
data    LNTLT0150608;LT0150608;EP2PMeter;6_D1;AURORA;L...
Name: 2, dtype: object
Sync_Electric_Meter_LNTLT0150608_NBPDCL_20240405_122504-103.xml
C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE\METER_XML_OUTPUT_M_S_&_MS_LINK\Sy

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_1436\2261353155.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  semicolon_split_for_file_name = rows[0].split(';')
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_1436\2261353155.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  semicolon_split = rows[0].split(';')


In [ ]:
# XML CODE WITH OUTPUT SAVE IN DIFF FILE
import pandas as pd
EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\New folder\Importation File\3PH\EESL_GENUS_PO0000000509_00000_EP2PMeter_20240405_075800_NBPDCL.csv")
EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.head()

,meter_data
0,LNTLT8003322;LT8003322;EP2PMeter;7_D2;REGOR;LN...


In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime
import shutil
import os

def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")


CURRENT_TIME = datetime.now().strftime('%Y%m%d_%H%M%S')

num_folder = 0
k = 100
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\XML_OUTPUT_FILE\METER,SIM AND METER-SIM LINK\METER_FILE'
new_folder_name = 'METER_XML_OUTPUT_M_S_&_MS_LINK'


new_folder_path = os.path.join(base_path, new_folder_name)
shutil.rmtree(new_folder_path)
os.makedirs(new_folder_path)

print(EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.head())

for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    print(rows)
    semicolon_split_for_file_name = rows[0].split(';')
    semicolon_split = rows[0].split(';')
            
    CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')


                                          meter_data
0  LNTLT8003322;LT8003322;EP2PMeter;7_D2;REGOR;LN...
meter_data    LNTLT8003322;LT8003322;EP2PMeter;7_D2;REGOR;LN...
Name: 0, dtype: object


C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_1436\2119018298.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  semicolon_split_for_file_name = rows[0].split(';')
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_1436\2119018298.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  semicolon_split = rows[0].split(';')


# create multiple single file

In [ ]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\EESL_GENUS_PO0000000100_00000_SIM_Meter_Link_20240115_145223_NBPDCL.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

num_folder = 0
new_folder_name = "EESL_GENUS_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    #print(f"single_row_data : {single_row_df}")
    last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
    #print(f"last_word : {last_part_after_semicolon}")


    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_GENUS_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_GENUS_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{last_part_after_semicolon}.csv'
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    
print("METER-SIM LINK FILE DONE")

: 

: 

: 

# change into zip file
###### C:\Users\Ratan Kumar Jha\Desktop\EESL_GENUS_PO0000000\data_insert_single_file

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


: 

: 

: 

In [ ]:
def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")

CURRENT_TIME_STAMP = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    semicolon_split = rows[0].split(';')
    print(semicolon_split[3])
    break

: 

In [ ]:
# xml code

: 

In [ ]:
for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    for i in rows:
        last_after_semicolon = rows[0].rsplit(';', 1)[-1]
        print(last_after_semicolon)
    break

: 

In [ ]:
for index,rows in EESL_GENUS_PO0000000403_00000_EP2PMeter_SBPDCL_Meter_Link_M_S_AND_MS_LINK.iterrows():
    
    last_after_semicolon = rows[0].split(';')
    print(last_after_semicolon[0])
    break
    
    
    
    

: 

: 

: 

# sim link file

In [ ]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\EESL_GENUS_PO0000000100_00000_SIM_20240104_104844_NBPDCL.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

num_folder = 0
new_folder_name = "EESL_BSNL_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    #print(f"single_row_data : {single_row_df}")
    last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
    #print(f"last_word : {last_part_after_semicolon}")


    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_BSNL_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_BSNL_PO0000000{k}_00000_SIM_{CURRENT_TIME_STAMP}_{last_part_after_semicolon}.csv'
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    
print("SIM LINK FILE DONE")

: 

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")

: 

: 

#### METER,SIM AND METER-SIM LINK
    - METER FILE PREPARE
    - SIM FILE PREPARE
    - SIM-METER LINK

#### SIM AND METER-SIM LINK FILE
    - SIM FILE
    - METER-SIM LINK

#### METER-SIM LINK
    - METER-SIM LINK



: 